In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline



from keras.layers import Input, Dense, Dropout, Flatten, Embedding, merge
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
#from tensorflow.keras import regularizers
from keras.regularizers import l2, l1_l2

In [ ]:
dr = pd.read_csv("../input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv")

In [ ]:
dr.head()

In [ ]:
simple_df_1 = pd.DataFrame()
simple_df_1[["team1", "team2", "Season"]] =dr[["WTeamID", "LTeamID", "Season"]].copy()
simple_df_1["pred"] = 1

simple_df_2 = pd.DataFrame()
simple_df_2[["team1", "team2", "Season"]] =dr[["LTeamID", "WTeamID", "Season"]]
simple_df_2["pred"] = 0

simple_df = pd.concat((simple_df_1, simple_df_2), axis=0)
simple_df.head()



In [ ]:
n = simple_df.team1.nunique()
n

In [ ]:
trans_dict = {t: i for i, t in enumerate(simple_df.team1.unique())}
simple_df["team1"] = simple_df["team1"].apply(lambda x: trans_dict[x])
simple_df["team2"] = simple_df["team2"].apply(lambda x: trans_dict[x])
simple_df.head()

In [ ]:
train = simple_df.values
#np.random.shuffle(train)

In [ ]:
train

In [ ]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype="int64", name=name)
    return inp, Embedding(n_in, n_out, input_length=10)(inp) 

def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [ ]:
n_factors = 50

team1_in, t1 = embedding_input("team1_in", n, n_factors, 1e-4)
team2_in, t2 = embedding_input("team2_in", n, n_factors, 1e-4)
#season_in, t3 = embedding_input("season_in", n, n_factors, 1e-4)

b1 = create_bias(team1_in, n)
b2 = create_bias(team2_in, n)
#b3 = create_bias(season_in, n)

In [ ]:
x = tf.keras.layers.Dot(axes=1)([t1, t2])
x = Flatten()(x)
x = tf.keras.layers.Add()([x, b1])
x = tf.keras.layers.Add()([x, b2])
#x = tf.keras.layers.Add()([x, b3])
x = Dense(1, activation="sigmoid")(x)
model = Model([team1_in, team2_in], x)
# starter_learning_rate = 0.001
# end_learning_rate = 0.0005
# decay_steps = 10000
# lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
#     starter_learning_rate,
#     decay_steps,
#     end_learning_rate,
#     power=0.5)
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.001
)


model.compile(optimizer, loss="binary_crossentropy")

In [ ]:
model.summary()

In [ ]:
history = model.fit([train[:, 0], train[:, 1]], train[:, 3],validation_split=0.3, batch_size=64, epochs=50, verbose=2)

In [ ]:
plt.plot(history.history["loss"])
plt.show()

In [ ]:
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
sub = pd.read_csv("../input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv")
sub

In [ ]:
sub["ID"].apply(lambda x: trans_dict[int(x.split("_")[1])])

In [ ]:
sub["team1"] = sub["ID"].apply(lambda x: trans_dict[int(x.split("_")[1])])
sub["team2"] = sub["ID"].apply(lambda x: trans_dict[int(x.split("_")[2])])
sub.head()

In [ ]:
sub["Pred"] = model.predict([sub.team1, sub.team2])
sub = sub[["ID", "Pred"]]
sub.head()

In [ ]:
sub.to_csv("MMLM22_submission.csv", index=False)

# **Reference:**
### [CF starter with Keras ~0.560136](https://www.kaggle.com/aikinogard/cf-starter-with-keras-0-560136/data)
